# Results for random forest classifier

## Multilabel classification with imbalanced data
### Confusion matrices

In [ ]:
"""Results for random forest classifier

Multilabel classification with imbalanced data
Confusion matrices
"""

In [1]:
import pandas as pd  # import libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix

# import data
df = pd.read_csv('data/SCADA_downtime_merged.csv', skip_blank_lines=True)

list1 = list(df['turbine_id'].unique())  # list of turbines to plot
list1 = sorted(list1, key=int)  # sort turbines in ascending order
list2 = list(df['TurbineCategory_id'].unique())  # list of categories
list2 = [g for g in list2 if g >= 0]  # remove NaN from list
list2 = sorted(list2, key=int)  # sort categories in ascending order
# categories to remove
list2 = [m for m in list2 if m not in (1, 12, 13, 14, 15, 17, 21, 22)]
list4 = list(range(0, 14))

for x in list1:  # filter only data for turbine x
    dfx = df[(df['turbine_id'] == x)].copy()
    for y in list2:
        # copying fault to new column (mins)
        # (fault when turbine category id is y)

        def ff(c):
            if c['TurbineCategory_id'] == y:
                return 0
            else:
                return 1

        dfx['mins'] = dfx.apply(ff, axis=1)

        dfx = dfx.sort_values(by='timestamp', ascending=False)
        # sort values by timestamp in descending order
        dfx.reset_index(drop=True, inplace=True)  # reset index

        if dfx.loc[0, 'mins'] == 0:
            # assigning value to first cell if it's not 0
            dfx.set_value(0, 'mins', 0)
        else:
            dfx.set_value(0, 'mins', 999999999)

        for i, e in enumerate(dfx['mins']):
            # using previous value's row to evaluate time
            if e == 1:
                dfx.at[i, 'mins'] = dfx.at[i - 1, 'mins'] + 10

        dfx = dfx.sort_values(by='timestamp')  # sort in ascending order
        dfx.reset_index(drop=True, inplace=True)  # reset index
        dfx['hours'] = dfx['mins'].astype(np.int64)
        # convert to hours, then round to nearest hour
        dfx['hours'] = dfx['hours'] / 60
        dfx['hours'] = round(dfx['hours']).astype(np.int64)

        def f11(c):  # > 48 hours - label as normal (9999)
            if c['hours'] > 48:
                return 9999
            else:
                return c['hours']

        dfx['hours'] = dfx.apply(f11, axis=1)

        def f22(c,):
            # filter out curtailment - curtailed when turbine is pitching
            # outside 0 deg <= normal <= 3.5 deg
            if (0 <= c['pitch'] <= 3.5 or c['hours'] != 9999 or (
                    (c['pitch'] > 3.5 or c['pitch'] < 0) and (
                        c['ap_av'] <= (0.1 * dfx['ap_av'].max())
                        or c['ap_av'] >= (0.9 * dfx['ap_av'].max())))):
                return 'normal'
            else:
                return 'curtailed'

        dfx['curtailment'] = dfx.apply(f22, axis=1)

        def f3(c,):
            # filter unusual readings, i.e. for normal operation,
            # power <= 0 in operating wind speeds, power >100...
            # before cut-in, runtime < 600 and other downtime categories
            if c['hours'] == 9999 and ((
                        3 < c['ws_av'] < 25 and (
                            c['ap_av'] <= 0
                            or c['runtime'] < 600
                            or c['EnvironmentalCategory_id'] > 1
                            or c['GridCategory_id'] > 1
                            or c['InfrastructureCategory_id'] > 1
                            or c['AvailabilityCategory_id'] == 2
                            or 12 <= c['TurbineCategory_id'] <= 15
                            or 21 <= c['TurbineCategory_id'] <= 22))
                    or (c['ws_av'] < 3 and c['ap_av'] > 100)):
                return 'unusual'
            else:
                return 'normal'

        dfx['unusual'] = dfx.apply(f3, axis=1)

        def f4(c):  # round to 6 hour intervals
            if c['hours'] == 0:
                return 10
            elif 1 <= c['hours'] <= 6:
                return 11
            elif 7 <= c['hours'] <= 12:
                return 12
            elif 13 <= c['hours'] <= 18:
                return 13
            elif 19 <= c['hours'] <= 24:
                return 14
            elif 25 <= c['hours'] <= 30:
                return 15
            elif 31 <= c['hours'] <= 36:
                return 16
            elif 37 <= c['hours'] <= 42:
                return 17
            elif 43 <= c['hours'] <= 48:
                return 18
            else:
                return 19

        dfx['hours6'] = dfx.apply(f4, axis=1)

        def f5(c):  # change label for unusual and curtailed data (20)
            if c['unusual'] == 'unusual' or c['curtailment'] == 'curtailed':
                return 20
            else:
                return c['hours6']

        dfx['hours_%s' % y] = dfx.apply(f5, axis=1)

        dfx = dfx.drop('hours6', axis=1)  # drop unnecessary columns
        dfx = dfx.drop('hours', axis=1)
        dfx = dfx.drop('mins', axis=1)
        dfx = dfx.drop('curtailment', axis=1)
        dfx = dfx.drop('unusual', axis=1)

    # separate features from classes for classification
    features = [
        'ap_av', 'ws_av', 'wd_av', 'pitch', 'ap_max', 'ap_dev',
        'reactive_power', 'rs_av', 'gen_sp', 'nac_pos']
    classes = [col for col in dfx.columns if 'hours' in col]
    list6 = features + classes  # list of columns to copy into new df
    df2 = dfx[list6].copy()
    df2 = df2.dropna()  # drop NaNs
    X = df2[features]
    X = preprocessing.normalize(X)  # normalise features to values b/w 0 and 1
    Y = df2[classes]
    Y = Y.as_matrix()  # convert from pd dataframe to np array
    # cross validation using time series split
    tscv = TimeSeriesSplit(n_splits=5)

    rf = RandomForestClassifier(criterion='entropy', n_jobs=-1)
    for train_index, test_index in tscv.split(X):
        # looping for each cross validation fold
        # split train and test sets
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        rf1 = rf.fit(X_train, Y_train)  # fit to classifier and predict
        Yp = rf1.predict(X_test)
        for m in list4:
            Yt = Y_test[:, m]
            Ypr = Yp[:, m]
            print(
                'Confusion matrix for turbine %s, turbine category %s'
                % (x, m))
            print(confusion_matrix(Yt, Ypr))
        print('------------------------------------------------------------')

Confusion matrix for turbine 1, turbine category 0
[[16269    42]
 [  252   792]]
Confusion matrix for turbine 1, turbine category 1
[[    0     0     0     0     0     0     0     0     0     5     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0 15978    41]
 [    0     0     0     0     0     0     0     0     0   251   792]]
Confusion matrix for turbine 1, turbine category 2
[[16269    42]
 [  252   792]]
Confusion matrix for turb

[[  43   41   11    4    0    5    4    1    0   14    0]
 [  34  383  176   77   25   10   17   15    3  141   24]
 [  23  222  113   55   42   18   16   22   11  188   24]
 [  31  175   91   22   44   18   30   13    7  245   10]
 [  14  160   79   44   30   11   27   11    5  234    7]
 [   7  103   79   49   41   19    9   23    4  275    3]
 [  16   64   80   43   44   20   17    8   21  274    1]
 [  14   86   83   37   34   12   12   13   14  269    2]
 [  10  103   60   49   24   20    6   10    8  228    9]
 [  35 1131  761  692  405  286  247  242  190 6193   14]
 [ 605  148  173  155   46   43    9   44   14  252  297]]
Confusion matrix for turbine 1, turbine category 9
[[    0     0     0     0     0     0     0     0     0    85   579]
 [    0     0     0     0     0     0     0     0     0   386    33]
 [    0     0     0     0     0     0     0     0     0   351     9]
 [    0     0     0     0     0     0     0     0     0   318    42]
 [    0     0     0     0     0   

[[   0 2624 6139]
 [   0 7161   30]
 [   0  407  994]]
------------------------------------------------------------------------
Confusion matrix for turbine 1, turbine category 0
[[13524    55]
 [  199  3577]]
Confusion matrix for turbine 1, turbine category 1
[[    0     0     0     0     0     0     0     0     0     5    79]
 [    0     1     0     0     0     0     0     0     0    35     0]
 [    0     0     0     0     0     0     0     0     0    34     2]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    32     4]
 [    0     0     0     0     0     0     0     0     0    34     2]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    33     3]
 [    1     6     4    16     1     4     3     1     2 13207    48]
 [   12    34    33     0     0     1     0     2

[[   16     0     0     0     0     0     0     0     0    10   289]
 [    0     0     0     0     0     0     0     0     0    35    37]
 [    0     0     0     0     0     0     0     0     0    63     9]
 [    0     0     0     0     0     0     0     0     0    47    25]
 [    0     0     0     0     0     0     0     0     0    35    37]
 [    0     0     0     0     0     0     0     0     0    36    36]
 [    0     0     0     0     0     0     0     0     0    37    35]
 [    0     0     0     0     0     0     0     0     0    40    32]
 [    0     0     0     0     0     0     0     0     0    61    11]
 [  113     0     0     0     0     0     0     0     0 13069    42]
 [  111     0     0     0     0     0     0     0     0   181  2948]]
------------------------------------------------------------------------
Confusion matrix for turbine 1, turbine category 0
[[13966    34]
 [  270  3085]]
Confusion matrix for turbine 1, turbine category 1
[[    0     0     2     0     0   

[[    0     0     0     0     0     0     0     0     0     2     2]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [  148     0     1     1     0     0     0     1     0 13529    35]
 [   64     0     0     0     5     0     0     0     0   262  3017]]
------------------------------------------------------------------------
Confusion matrix for turbine 2, turbine category 0
[[    0     0     0     0     0     0     0     0     0     2     2]
 [    0     0     0     0     0     0     0    

...

[[    0     0     0     0     0     0     0     0     0     3    25]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    31     0]
 [    6    23     6     1     5     9     4     0     0 14412   367]
 [  724     0     1     0     0     0     0     0     1   253  5661]]
Confusion matrix for turbine 4, turbine category 4
[[14747   370]
 [  254  6413]]
Confusion matrix for turbine 4, turbine category 5
[[    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0 

[[    2     1     0     0     0     0     0     0     0     6    20]
 [    0     0     0     0     1     0     0     0     0    71     0]
 [    0     0     0     0     0     0     0     0     0    71     1]
 [    0     0     0     0     0     0     0     0     0    68     4]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0     0     0     0     0    71     1]
 [    0     0     0     0     0     0     0     0     0    71     1]
 [    0     0     0     0     0     0     0     0     0    68     4]
 [    0     1     0     0     0     1     0     0     0    67     3]
 [    1    16    11     7     2     2     3     4     1 16995   173]
 [    5     0     0     9     0     2     0     6     0   225  3717]]
Confusion matrix for turbine 4, turbine category 6
[[  300     0     0     0     0     0     0     0     0     6    14]
 [   36     0     0     0     0     0     0     0     0    57    15]
 [   36     0     0     0     0     0     0     0  

[[    0     0     0     0     0     0     0     0     0     5    48]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    6    23    16     5     3     4     5     6     4 20093   166]
 [   19     1     1     3     1     0     0     0     5   128   954]]
Confusion matrix for turbine 4, turbine category 7
[[    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0  

[[  136     6     0     0     0     0     0     0    41   110   857]
 [    7     0     0     1     0     0     0     0     1   530    13]
 [    1     0     0     0     0     0     0     0     0   533     6]
 [    0     0     0     0     0     0     0     0     1   529     2]
 [    0     0     0     0     0     0     0     0     1   499     4]
 [    0     0     0     0     1     0     0     0     2   500     1]
 [    1     0     0     0     0     0     0     0     0   457    18]
 [    0     1     0     0     0     0     0     0     0   458     9]
 [    1     0     0     0     0     0     0     0     1   423    11]
 [    6     7     4    15    13     7     1     2    12 16220    53]
 [   20     0     0     0     0     0     0     0     0    37   225]]
Confusion matrix for turbine 4, turbine category 10
[[20300    60]
 [   82  1342]]
Confusion matrix for turbine 4, turbine category 11
[[    0     0     0     0     0     0     0     0     0    20    10]
 [    0     0     0     0     0     

KeyboardInterrupt: 

[[ 202  186   75  126   29   13    4    3    1   23   48]
 [ 172 1369  475  382  143   66   32   21   26  233    0]
 [  59  668  351  216  115   47   23   24   18  276    1]
 [  30  462  201  137  128   69   34   28   13  256    6]
 [  19  306  244  115   84   18   31   15   15  269   10]
 [  14  185  193  102   90   31   33   32   17  239    1]
 [  10  113  112   65   55   21   22   20   29  332    7]
 [   7  102  121   86   41   29   31   31   18  274    1]
 [  16   79  143   90   56   27   21   17   30  186    7]
 [ 201  863  857  526  316  157  287  205  295 2348   19]
 [  31   19   31   54    8    6   15    5   15   38    6]]
[[  37   53    8    8    1    4    4    1    0    7    0]
 [  30  366  176   97   26   10   18   12    1  159   10]
 [  18  232   93   76   39   17   18   13    7  206   15]
 [  24  176   86   27   31   15   32   12    6  257   20]
 [  19  160   60   57   35   11   19    5    4  247    5]
 [  11  103   76   53   43   21   16   11    3  274    1]
 [   9   56  

[[  207     0     0     0     0     1     0     0     0    41   137]
 [    0     0     0     0     0     0     0     0     0   144     0]
 [    0     0     0     0     0     0     1     0     0   143     0]
 [    0     0     0     0     0     0     0     1     0   141     2]
 [    0     0     0     0     0     0     0     0     0   142     2]
 [    0     0     0     0     0     0     0     0     0   140     4]
 [    0     0     0     0     0     0     0     0     0   141     3]
 [    1     0     1     0     2     0     0     0     0   134     5]
 [    0     0     0     0     0     0     0     0     0   108     0]
 [    6    53    29    38    19     6    23    19     9 19054   116]
 [   43    21    12    10     4     1     2     2     3    98   682]]
[[    0     0     0     0     0     0     0     0     0     3     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     1     0     0     0    33     2]
 [    0     0     0     0     0  

[[   94     0     1     0     0     0     0     1     2    90    65]
 [    0     0     0     2     1     0     1     2     2   170     2]
 [    0     1     0     1     0     2     0     1     0   173     1]
 [    0     0     0     0     0     0     0     0     0   143     1]
 [    0     0     0     0     1     0     0     0     0   142     1]
 [    0     0     0     0     0     3     0     0     0   140     1]
 [    0     1     0     0     0     0     0     0     1   139     3]
 [    0     0     0     0     0     0     0     0     0   142     2]
 [    0     0     0     0     0     0     0     0     1   139     4]
 [   19   103    32    45    54    34    58    15    15 18580    38]
 [    2     0     0     0     1     2     0     1     1   106   204]]
[[    0     0     0     0     0     0     0     0     0    71     4]
 [    0     0     0     3     0     0     0     0     0    66    39]
 [    0     0     0     0     1     0     0     0     0    62    45]
 [    0     0     0     0     0  

[[  136     0     3     0     0     0     0     0     0    26   106]
 [    1     1     0     0     0     0     0     0     0   148    30]
 [    0     0     0     0     0     0     0     0     2    79    36]
 [    0     0     0     0     0     0     0     0     0    71    37]
 [    0     0     0     0     0     0     0     0     0    70    38]
 [    9     0     0     0     0     0     0     0     0    78    21]
 [    0     0     0     0     0     0     0     0     0   107     1]
 [    3     0     0     0     0     1     0     0     0   101     3]
 [    3     0     0     0     0     0     0     0     0   104     1]
 [  310    11    24    17     0    23     8     3     2 11386   109]
 [ 2441     2     5     1     3     6     4    36    19   242  1711]]
[[    8     0     0     0     0     0     0     0     0     4     2]
 [   36     0     0     0     0     0     0     0     0    36     0]
 [   28     0     0     0     0     0     0     1     0    36     7]
 [   20     0     0     0     0  

[[   79     1     1     0     0     0     0     0     0    11    18]
 [    4     4     4     0     0     0     0     0     0   126     6]
 [    3     1     0     0     0     0     0     0     0   118     2]
 [    1     2     1     0     0     0     0     0     0   103     1]
 [    0     2     0     0     0     0     0     0     0   106     0]
 [    0     1     0     1     0     0     0     0     0   106     0]
 [    0     0     0     0     0     0     0     0     0   108     0]
 [    0     0     0     0     0     0     0     0     0    84     0]
 [    0     1     0     1     1     0     0     0     3    65     1]
 [    6    24    25    17    18    24     9    31    20 20111    86]
 [   20     0     1     0     0     1     1     1     0    79   405]]
[[    1     0     0     0     0     0     0     0     0     7    38]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0  

...

[[ 786   61    2    0    1    5    3    6    0  364  556]
 [   5   92    8   19    5   16    7    7    3 1276   76]
 [   0  100    8    5    7    4    2    1    2 1107   51]
 [   1   50    5    4    2    5    2    0    1  872  108]
 [  24   38   15    5    1    3    4    1    2  654   48]
 [   4   39    0    1    0    1    0    0    1  447   40]
 [   0   22    0    0    0    0    1    0    0  445   29]
 [   0    1    0    0    1    1    1    0    0  390   54]
 [   0    0    0    0    2    1    0    0    0  373   25]
 [   4  232   24   19   22   31   19    5   15 8575  188]
 [ 263  479    1    0   96   84    1    1    1  381 1419]]
[[ 329    5    1    0    2    0    2    4    1   59  133]
 [  42   46   22    5    3   11    8    2    0  488   23]
 [  35   60   19    9    8    3    4    2    0  393   36]
 [   2   24   12    7    7    5    8    1    1  378   31]
 [   4   23   23   10    8    5    3    3    2  359   20]
 [   2   13   18    4   13    2   11    2    2  262   18]
 [  16   18  

[[    0     0     0     0     0     0     0     0     0    14   117]
 [    0     0     0     0     0     0     1     0     0   111    32]
 [    0     1     0     0     0     0     0     0     0   119    24]
 [    0     2     0     0     0     0     0     0     0   132    10]
 [    0     0     0     0     0     0     0     0     0   139     5]
 [    3     0     0     0     0     0     0     0     0   122    19]
 [    0     0     0     0     0     0     0     0     0   140     4]
 [    0     0     0     0     0     0     0     0     0   141     3]
 [    0     0     0     0     0     0     0     0     0   107    37]
 [   14     5     1     3     5     6     8     0     0 10044   113]
 [  127     7     1     0     1     1     1     0     0  1467  8576]]
[[   47     4     0     1     0     1     0     2     0   405  1020]
 [    1     1     0     0     0     1     0     0     0   852    88]
 [    3     0     1     0     0     7     0     2     1   476   128]
 [    1     2     1     1     0  

[[    6     0     0     0     0    17     0     1     0    84   306]
 [    0     0     0     0     0     0     0     0     0   244     8]
 [    0     0     0    11     0     0     0     0     0   240     1]
 [    0     1     0     5     0     0     0     0     0   246     0]
 [    0     0     0     4     0     0     0     1     2   222     1]
 [    0     0     0     4     0     0     0     1     0   209     2]
 [    0     0     0     0     0     0     0     0     0   215     1]
 [    0     0     0     0     0     0     0     0     0   214     2]
 [    0     0     0     0     0     0     0     0     0   215     0]
 [    2     7    22    31     1    13    25     7    26 18420    83]
 [    5     1     0     0     1     2     1     2     0   147   759]]
[[    0     0     0     0     0     0     0     0     0     3     1]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0  

[[    0     0     0     0     0     0     0     0     0     5     3]
 [    0     0     0     0     0     0     0     0     0    66     6]
 [    0     0     1     0     0     0     0     0     0    68     3]
 [    0     0     0     0     0     0     0     0     0    71     1]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     1     0     0     0     0     0    71     0]
 [    0     0     0     0     0     0     0     0     0    71     0]
 [    0     0     0     0     0     0     0     0     0    69     2]
 [    0     0     0     0     0     0     0     0     0    71     0]
 [   15     4     7     2     7     0     2     3     2 14339    32]
 [   74    27    31     2     3    27     0     1     3    86   839]]
[[  105     2     0    11    36     3    14     4     1    76   180]
 [   22     1     0     1     0     2     4     0     2   176    32]
 [    7     3     1     4     0     0     1     0     0   143    37]
 [    3     0     2     5     0  

[[   89     0     0     1     5     1     0     0     0   125    22]
 [    3     0     1     0     0     2     0     0     0   136     7]
 [    0     0     0     1     0     0     0     0     1   143     1]
 [    0     0     0     1     1     0     0     0     0   143     3]
 [    0     0     0     4     1     0     0     0     0   146     0]
 [    0     0     0     0     0     0     0     0     1   153     1]
 [    4     0     0     0     0     0     0     0     0   147     4]
 [    0     0     0     1     0     0     0     0     0   154     3]
 [    1     0     0     4     1     0     0     0     0   135     2]
 [  206    18    25    41    22    14    53    12    49 14465    16]
 [  134     6     1     6     0     5     0     2     0   725   278]]
[[    4     0     0     0     0     0     0     0     0     7    14]
 [    0     0     0     0     0     0     0     0     0    32    65]
 [    0     0     0     0     0     0     0     0     0    34    63]
 [    0     0     0     0     0  

[[    0     0     0     0     0     0     0     0     0     7     0]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0     0     0     0     0    71     1]
 [    0     0     0     0     0     0     0     0     0    71     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    15    21]
 [    0     0     0     0     0     0     0     0     0 15854    54]
 [    0     1     0     0     0     0     0     1     0   240   767]]
[[    0     0     0     0     0     0     0     0     0    55   169]
 [    0     0     0     0     0     0     0     0     0   256     4]
 [    0     0     0     2     0     0     0     0     0   223     1]
 [    0     0     0     0     0  

[[    1     0     0     0     0     0     0     0     0     0     3]
 [    5     1     0     0     0     0     0     0     0     0    30]
 [    1     2     0     0     0     0     0     0     0    30     3]
 [    0     0     0     0     0     0     0     0     0    36     0]
 [    0     0     0     0     0     0     0     0     0    35     1]
 [    0     0     0     0     0     0     0     0     0     1    35]
 [    0     0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0     3    33]
 [    0     0     0     0     0     0     0     0     0    30     6]
 [    0     3    11     7     1    15     8     1    17 18238   231]
 [   30    39    38     6    19    13     2     0     1   163  2616]]
[[    0     0     0     0     0     0     1     0     0    28    27]
 [    0     0     0     0     0     0     0     0     0   100     8]
 [    0     0     0     0     0     0     0     0     0    94    14]
 [    0     0     0     0     0  

[[    3     0     0     0     0     0     0     0     0    16    22]
 [    0     0     0     0     0     0     0     0     0   107     1]
 [    0     0     0     0     0     1     0     0     0   107     0]
 [    0     0     0     0     0     1     0     0     0   106     1]
 [    0     0     0     0     0     0     0     0     0   108     0]
 [    0     0     0     0     0     0     0     0     0    77     1]
 [    0     0     0     0     0     0     0     0     0    72     0]
 [    1     0     0     0     0     0     0     0     0    69     2]
 [    0     0     1     0     0     0     0     0     0    71     0]
 [   32    20    16    14    11    27     7     6     2 17320    36]
 [   42     0     0     0     0     0     0     0     0   278  2208]]
[[    1     0     0     0     0     0     0     0     0    47   124]
 [    0     0     0     0     0     0     0     0     0   301    10]
 [    0     0     0     0     1     0     0     0     0   163    23]
 [    0     0     0     0     0  

[[    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [   11    44    26    15    20    25    15    28    40 18776   166]
 [   23     6     4     6     3     6     5    11     1   232  1460]]
[[    0     0     0     0     0     0     0     0     0    20    16]
 [    0     0     0     0     0     5     0     0     0   129    11]
 [    0     0     0     0     0     0     0     0     0   131    13]
 [    0     0     0     0     0  

[[    1     0     0     0     0     0     0     0     0    25    16]
 [   21     0     6     0     0     0     0     0     0    85    32]
 [    0     0     0     0     0     0     0     0     0   100    44]
 [    0     0     0     0     3     3     1     0     0   125    12]
 [    0     0     8     5    12     1     1     0     0   100    17]
 [    0     0     0     1     1     1     1     0     1   103    36]
 [    0     0     0     0     0     0     0     0     1   135     8]
 [    0     0     0     1     0     0     0     0     0   127    16]
 [    0     0     0     3     0     0     1     0     1   126    13]
 [    0     5   111   243    59    36   128    23    38 17526   641]
 [    2    16    13     5     3     2     4     1     0   316  1479]]
[[  103    42     0     0     0     0     0     0     0     7   190]
 [    0     0     4     0     0     0     0     0     0    61    18]
 [    0     0    18     8     0     0     0     0     0    46     0]
 [    0     0     4     2     0  

...

[[    0     0     0     0     0     0     0     0     0    90    17]
 [    0     0     0     0     0     0     0     0     0   160    20]
 [    0     0     0     0     0     0     0     0     0   147    33]
 [    0     0     0     0     0     0     0     0     0   123    57]
 [    0     0     0     0     0     0     0     0     0   119    57]
 [    0     0     0     0     0     0     0     0     0    92    52]
 [    0     0     0     0     0     0     0     0     0    89    72]
 [    0     0     0     0     0     0     0     0     0   106    74]
 [    0     0     0     0     0     0     0     0     0   120    60]
 [    0     0     0     0     0     0     0     0     0 14170   405]
 [    1     0     0     0     0     0     0     0     0  1388  2717]]
[[    0     0     0     0     0     0     0     0     0   201  1862]
 [    0     0     0     0     0     0     0     0     0     6   144]
 [    0     0     0     0     0     0     0     0     1    23   120]
 [    0     0     0     0     0  

[[ 139    0    0    0    0    1    2    0    0   37  212]
 [  19    0    1    0    0    0    0    0    0  121    9]
 [   1    3    0    0    1    2    0    0    0   60   41]
 [  12    0    0    1    0    0    1    0    0   68   26]
 [  23    2    0    0    0    1    0    0    0   69   13]
 [  13    0    0    0    0    0    0    0    0   45   50]
 [  14    7    0    0    0    0    0    0    0   17   70]
 [  10    5    0    1    1    0    0    0    0   23   68]
 [   2    2    2    0    0    0    2    0    0   97    3]
 [ 340  117   38   74   26   26   80   79   26 9922   98]
 [2142   78   66    2  105   21   21   64   16  755 6270]]
[[  174     0     0     0     3     2     0     0     0    60    61]
 [    2     1     0     0     0     0     0     0     0   249    18]
 [    1     1     0     0     0     1     0     7     2   103    14]
 [    0     1     0     1     1     0     0     0     0    97     8]
 [    0     0     2     0     0     0     0     0     0    94     9]
 [    5     1   

[[   95     0     0     0     0     7     1     0     0   302    82]
 [    0     0     0     0     0     0     0     0     0   332    47]
 [    0     0     0     0     0     0     0     0     0   332    28]
 [    0     0     0     0     0     0     0     0     0   352     8]
 [    0     0     0     0     0     0     0     0     0   347    13]
 [    0     0     0     0     0     0     0     1     0   338    21]
 [    0     0     0     0     1     0     0     0     0   354     5]
 [    0     0     0     0     0     0     0     0     0   354     6]
 [    0     0     0     0     0     0     0     1     0   347    12]
 [    2     2     1     1     0     4     6     2     1 18066    30]
 [    0     0     0     0     1     0     0     0     1    83   232]]
[[    0     0     0     0     0     0     0     0     0    83    15]
 [    0     0     0     0     0     0     2     0     0    87     0]
 [    0     0     1     0     0     0     0     0     0    68     3]
 [    0     0     0     0     0  

[[  378     0     0     0     0     0     0     0     0    49   158]
 [   14     0     2     0     0     0     0     0     0    34    96]
 [   13     6     0     8     1     0     0     0     0    85    18]
 [    0     7     1     0     1     0     0     0     0    92     0]
 [    0     8     1     0     0     0     0     0     0   110     2]
 [    0     8     2     0     0     0     0     0     0   107     3]
 [    0     4     9     9     0     0     0     0     0    95     2]
 [    0     2     0     0     0     0     0     0     0   107     8]
 [    1     6     2    13     0     0     0     0     0    72     5]
 [   36   275   281   434    58     0     0     0     0 12727   129]
 [  111     2     0     3    11     0     0     0     0   173   851]]
[[    0     0     0     0     0     0     0     0     0     0     0]
 [   17     0     0     0     0     0     0     0     0     4    10]
 [   15     2     0     0     0     0     0     0     0     0    10]
 [    5     8     0     0     0  

[[    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [  142   361   136   170    41    32    15    45    23 15280    82]
 [  306    21     1     1     0     6     0     2     4   173   299]]
[[    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0  

[[ 278  194  114  180    9    8    3    1    1   20]
 [ 158 1385  725  427  166   63   30   12   12  240]
 [  65  659  383  285  132   75   31   15    7  276]
 [  39  461  279  165   93   50   35   15    7  224]
 [  17  236  212  134   84   26   21   14    9  266]
 [  14  156  124   93   70   29   21   30   18  219]
 [   2   78   84   67   54   14   13   17   28  285]
 [   9   80   84   75   40   20   23   17   27  222]
 [   9   96   96   56   51   21   15   11   31  175]
 [  58  624  539  371  293  132  179   89  246 1915]]
[[  45   59   13    9    2    3    3    4    0   15]
 [  50  466  175  105   39   25   17   13    6  214]
 [  28  267  146   72   54   22   12   37   13  224]
 [  15  206  117   53   53   20   26   32   16  289]
 [   8  183  102   58   42   20   30   14    6  303]
 [  15  146  101   59   56   30   13   29   10  293]
 [  19   99   98   47   47   26   12    9   23  307]
 [  32  130   83   47   58   24    7   10   16  277]
 [  14  106   90   55   34   39   17   11    

[[  806     8     0     1     8     1     0     2     0   209]
 [  164    25     4     4    14     4     3     1     1   412]
 [   32     3     1    22     8     2     1     0     0   419]
 [    7     8    15     0     0     0     0     4     0   434]
 [   25    16     1     5     5     8     2     1     0   382]
 [   20    45     1     1     6     2    10     0     1   346]
 [   14    33     3     0     8     2     8     4     0   360]
 [    8     5     2     0     2     1     8     5     1   400]
 [   34    10     6     2     8     1     0     1     0   370]
 [  183    57    62   262    71    32   107    46    73 12727]]
[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     

[[   10     0     0     0     0     0     0     0     0     7]
 [   12     0     0     0     0     0     0     0     0    24]
 [    5     0     0     0     0     0     0     0     0    31]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    1     3     6     3     3    12     3     5     8 18742]]
[[    2     0     0     0     0     0     0     0     0     6]
 [   25     0     0     0     0     0     0     0     0    47]
 [   10     0     0     0     0     0     0     0     0    62]
 [    0     0     0     0     0     0     0     0     0    72]
 [    0     2     1     0     0     0     0     0     0    69]
 [    0     1     0     0     0     0     0     0     

[[    0     0     0     0     0     0     0     0     0    12]
 [    0     2     0     0     0     0     0     0     0   106]
 [    0     0     0     0     0     0     0     2     0   106]
 [    0     0     0     0     0     0     0     0     0   108]
 [    0     0     0     0     0     0     0     0     0   108]
 [    0     0     0     0     0     0     0     0     0   108]
 [    2     0     0     0     0     0     0     0     0   106]
 [    9     1     0     0     4     0     0     0     0    94]
 [    0     0     0     0    27     0     0     0     0    81]
 [    0     0     3     1     2     2     2     2     2 14866]]
[[   72     8     5     4     0     0    79    12     0   113]
 [    1     2     0     0     0     0     0     0     0    33]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     0    36]
 [    0     0     0     0     0     0     0     0     

[[  134     0     0     0     0     3     2    10     0  2467]
 [    6     0     1     0     0     1     0    11     0   720]
 [    8     0     2     1     0     0     1    17     0   487]
 [    9     0     0     0     3     0     0    35     1   414]
 [   24     0     0     0     3     1     9     9     1   350]
 [   30     0     0     0     0     0     6     5     0   291]
 [    1     0     0     0     0     0     0     1     0   283]
 [    3     0     0     0     1     0     0     2     0   246]
 [    0     0     0     0     0     0     0     0     0   252]
 [    0    11     3    13    23     3     2   143     6 13074]]
[[   13    19     0     0     0     0     0     0     0   138]
 [    0    12     0     2     0     0     0     0     0    58]
 [    5     2     4     1     0     0     0     0     0    60]
 [    3     4     1     0     0     0     0     0     0    64]
 [    3     3     0     0     0     0     0     0     0    66]
 [    4     3     4     1     0     0     0     0     

[[1214   55    5    3    1    3    1   20    2  453]
 [  11   92    6   17    2    6    3    8    8 1183]
 [  13  103    2    5    2    1    1    1    6 1009]
 [  17   50    4    3    0    0    1    0    1  854]
 [  38   37   19    1    0    0    0    0    2  604]
 [   7   37    0    3    3    0    0    0    0  411]
 [   1   17    0    0    0    0    0    0    0  407]
 [   5    1    0    0    0    1    0    0    0  369]
 [   2    0    1    1    1    0    0    0    0  324]
 [   9  167   22   14   16   21   16    7   13 8350]]
[[ 423   10    2    4    6    4    3   33    4   74]
 [  42   81   22    8    3   34   19   12    1  606]
 [  51   59   19   16   12   10    6    8    1  531]
 [  20   47   14   12    4    8    6    2    2  481]
 [   8   26   24   20    7    6    3    3    2  455]
 [   8   12   17   14   13    3   13    3    4  332]
 [  43   16   14    7    0    5    3    7    3  298]
 [  31    5   11    8    8    4    1    0    3  325]
 [  12   11    3    6    1    0    0    3    

[[  143    13     6     0     4     0     2     0     0    19]
 [   54    27    30     2     4     2     0     0     0   205]
 [   37     6    24     4     2     2     1     0     1   247]
 [    7     8     4     1    14     2     2     2     0   255]
 [   59     4     7     4     8     1     5     1     0   199]
 [   28     6     2     6    28     4     2     1     1   210]
 [    1     1     4     4    29     1     2     0     0   246]
 [   13     2    13     3     3     1     1     0     0   218]
 [   25     2     6     2     1     0     2     0     0   214]
 [   27   149   207    68    45    56    92    54    39 14013]]
[[    7     4     0     0     5     3     0     0     0    72]
 [    0     0     0     0     0     0     0     0     0   146]
 [    0     0     0     0     0     0     0     0     0   144]
 [    0     0     0     0     0     0     0     0     0   144]
 [    0     0     0     1     0     2     0     0     0   141]
 [    0     0     0     0     0     3     0     0     

[[   89     0     0     0     0     0     2     0     0   595]
 [   19     0     2     0     0     0     0     1     0   308]
 [   36     0     0     0     0     0     0     0     0   252]
 [   36     0     0     0     0     0     0     0     0   238]
 [   36     0     0     0     0     0     1     0     0   208]
 [   37     0     0     0     0     1     0     0     0   178]
 [   46     0     0     0     0     0     0     0     0   170]
 [   45     0     0     0     0     0     0     0     0   164]
 [   47     0     0     0     0     0     0     0     0   133]
 [    6     2     2    12     3     2    12     2     1 16008]]
[[  244     3     0     0    39    75    33     1     1   144]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     

[[    1     0     0     0     0     0     0     0     0     4]
 [    0     0     0     0     0     0     0     0     0    30]
 [    0     0     1     0     0     0     0     0     0    33]
 [    0     2     0     0     0     0     0     0     0    32]
 [    3     0     2     0     0     1     0     0     0    26]
 [    0     0     0     0     0     0     0     0     1    21]
 [    1     0     0     0     0     0     0     0     0    25]
 [    0     0     0     1     0     0     0     0     0    24]
 [    0     0     0     0     0     0     0     0     0    25]
 [   59    28    50    23    14    37     8    11     7 14101]]
[[    0     0     0     0     0     0     0     0     0     4]
 [    0     0     0     0     0     0     0     0     0    24]
 [    0     0     0     0     0     0     0     0     0    22]
 [    0     0     0     0     0     0     0     0     0    26]
 [    0     0     0     0     0     0     0     0     0    30]
 [    0     0     0     0     0     0     0     0     

[ 0.0933323   0.10989166  0.11766825  0.05614049  0.11224687  0.08126399
  0.12064259  0.05895795  0.13374344  0.11611245]
[ 0.09142572  0.11444904  0.11910066  0.06154669  0.10538376  0.08307471
  0.12178083  0.0603405   0.12457242  0.11832566]
[ 0.09373139  0.11561793  0.12068691  0.06018291  0.09371799  0.08434777
  0.11792445  0.06703002  0.12962012  0.11714051]
[ 0.08800064  0.11568187  0.11985847  0.06279341  0.10463351  0.08692242
  0.11101742  0.06704839  0.1209844   0.12305948]
[ 0.11250946  0.1168199   0.11550914  0.06283925  0.09098561  0.08410523
  0.11158497  0.0729696   0.10865319  0.12402364]
[ 0.0778841   0.11154581  0.12465396  0.07033192  0.06541114  0.0821813
  0.15298234  0.08171407  0.11007968  0.12321568]
[ 0.08259304  0.10839353  0.13567985  0.06213928  0.06444749  0.07195679
  0.15717343  0.06940598  0.1131782   0.13503241]
[ 0.0842396   0.11364558  0.13438868  0.06392707  0.07180858  0.06324969
  0.1601231   0.06166414  0.09731089  0.14964269]
[ 0.07504246  0.1

[ 0.06874151  0.10718303  0.1106965   0.06882545  0.06916366  0.07143073
  0.17031083  0.08974133  0.11341696  0.13049   ]
[ 0.06283533  0.11089429  0.12737674  0.06959678  0.06745864  0.06451782
  0.16699392  0.08329332  0.11349448  0.13353868]
[ 0.06403991  0.1318859   0.1120675   0.08975758  0.05901902  0.05874002
  0.1004157   0.14024448  0.11726996  0.12655994]
[ 0.06482403  0.12289132  0.10763163  0.06895969  0.05725932  0.06405451
  0.11920844  0.13971939  0.12906882  0.12638284]
[ 0.06712332  0.13477827  0.11630043  0.08909739  0.0629202   0.0753663
  0.10602372  0.0913916   0.14500018  0.11199858]
[ 0.06862771  0.14373003  0.12346359  0.08784386  0.06267372  0.06712294
  0.10066481  0.09284388  0.1261378   0.12689166]
[ 0.06943919  0.13313029  0.12401727  0.0982027   0.06381223  0.07076575
  0.10127271  0.09368987  0.12265383  0.12301614]
[ 0.08111898  0.1088717   0.13460148  0.05913963  0.06770224  0.077296
  0.14338437  0.09165023  0.09671588  0.1395195 ]
[ 0.07234426  0.122